In [0]:
import os
import logging
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when, length
from pyspark.sql.types import StringType, StructType, StructField
from pyspark.sql.utils import AnalysisException


# Inicializando a Spark Session
spark = SparkSession.builder.appName("load_tables").getOrCreate()

In [0]:


# Create the log directory if it doesn't exist
log_directory = '/FileStore/glassdoor/logs/'
if not os.path.exists(log_directory):
    os.makedirs(log_directory)

logger = logging.getLogger(__name__)
handler = logging.FileHandler(os.path.join(log_directory, 'extract.log'))
formatter = logging.Formatter('%(asctime)s - %(levelname)s - [Extract Files Log] %(message)s')
logger.info("Iniciando Spark Session - Extract Dataset and load into Bronze layer - Grupo 1")


In [0]:
# Função para verificar se o arquivo existe utilizando a API do Databricks
def file_exists(filepath):
    logger.info("cerificar se o arquivo existe utilizando a API do Databricks")
    try:
        return len(dbutils.fs.ls(filepath)) > 0
    except Exception as e:
        logger.error(f"Arquivo {e} não existe!")
        return False

# Função para ler um DataFrame com tratamento de exceções
def read_csv_with_options(spark, filepath, delimiter, encoding="utf-8", header=True):
    if file_exists(filepath):
        try:
            logger.info(f"Iniciando a leitura do DataFrame: {filepath} com tratamento de exceções")
            return spark.read.option("delimiter", delimiter) \
                             .option("header", str(header).lower()) \
                             .option("encoding", encoding) \
                             .csv(filepath)
        except AnalysisException as e:
            logger.error(f"Erro ao ler o arquivo {filepath}: {str(e)}")
        except Exception as e:
            logger.error(f"Erro inesperado ao ler o arquivo {filepath}: {str(e)}")
    else:
        logger.error(f"Arquivo não encontrado: {filepath}")
    return None

# Leitura dos arquivos
df_bronze_bancos = read_csv_with_options(spark, "/FileStore/Bancos/EnquadramentoInicia_v2.tsv", "\t")
df_bronze_reclamacoes = read_csv_with_options(spark, "/FileStore/Reclamacoes", ";", "cp1252")

In [0]:
# Definindo as colunas padrão esperadas para CNPJ e Segmento como conjuntos para facilitar operações de conjunto
colunas_padrao_CNPJ = set([
    'employer_name', 'reviews_count', 'culture_count', 'salaries_count',
    'benefits_count', 'employer_website', 'employer_headquarters',
    'employer_founded', 'employer_industry', 'employer_revenue', 'url',
    'Geral', 'Cultura e valores', 'Diversidade e inclusão',
    'Qualidade de vida', 'Alta liderança', 'Remuneração e benefícios',
    'Oportunidades de carreira', 'Recomendam para outras pessoas(%)',
    'Perspectiva positiva da empresa(%)', 'CNPJ', 'Nome', 'match_percent'
])

colunas_padrao_seg = set([
    'employer_name', 'reviews_count', 'culture_count', 'salaries_count',
    'benefits_count', 'employer_website', 'employer_headquarters',
    'employer_founded', 'employer_industry', 'employer_revenue', 'url',
    'Geral', 'Cultura e valores', 'Diversidade e inclusão',
    'Qualidade de vida', 'Alta liderança', 'Remuneração e benefícios',
    'Oportunidades de carreira', 'Recomendam para outras pessoas(%)',
    'Perspectiva positiva da empresa(%)', 'Segmento', 'Nome', 'match_percent'
])

# Colunas totais que incluem todas as possíveis de ambos os esquemas
all_columns = list(colunas_padrao_CNPJ | colunas_padrao_seg)

# Caminho para os arquivos de empregados no DBFS
path = "/FileStore/Empregados"

# Usando dbutils para listar arquivos
files = dbutils.fs.ls(path)

# Lendo cada arquivo e ajustando o esquema
df_list = []
logger.info(f"Iniciando a leitura do DataFrame: {path} com tratamento de exceções")
for file_info in files:
    file_path = file_info.path
    if file_path.endswith('.csv'):
        df = spark.read.option("delimiter", "|").option("header", "true").option("encoding", "utf-8").csv(file_path)
        
        try: 
            # Verificar colunas existentes e adicionar colunas faltantes
            existing_columns = set(df.columns)
            missing_columns = set(all_columns) - existing_columns  # Usar set para all_columns para garantir operação de conjunto
        except AnalysisException as e:
            logger.error(f"Erro ao Verificar colunas existentes e adicionar colunas faltantes: {str(e)}")

        for coluna in missing_columns:
            logger.info(f"O DataFrame: {file_path} encontrou arquivos com diferentes Schemas. Adicionar colunas faltantes.")
            df = df.withColumn(coluna, lit(None).cast(StringType()))
        
        # Selecionar as colunas na ordem padrão para uniformidade
        df = df.select(*all_columns)
        df_list.append(df)
try:
    # Unificar todos os DataFrames
    df_unificado = df_list[0]
    for df in df_list[1:]:
        df_unificado = df_unificado.unionByName(df)
        logger.info(f"Unificados todos os DataFrames.")
except AnalysisException as e:
            logger.error(f"Erro ao Unificar todos os DataFrames: {str(e)}")

# Exibir o esquema unificado e algumas linhas para verificar
df_unificado.printSchema()
display(df_unificado)

root
 |-- employer_name: string (nullable = true)
 |-- Cultura e valores: string (nullable = true)
 |-- employer_website: string (nullable = true)
 |-- Alta liderança: string (nullable = true)
 |-- culture_count: string (nullable = true)
 |-- employer_revenue: string (nullable = true)
 |-- Oportunidades de carreira: string (nullable = true)
 |-- Perspectiva positiva da empresa(%): string (nullable = true)
 |-- Geral: string (nullable = true)
 |-- salaries_count: string (nullable = true)
 |-- match_percent: string (nullable = true)
 |-- employer_founded: string (nullable = true)
 |-- employer_headquarters: string (nullable = true)
 |-- employer_industry: string (nullable = true)
 |-- CNPJ: string (nullable = true)
 |-- url: string (nullable = true)
 |-- Recomendam para outras pessoas(%): string (nullable = true)
 |-- Segmento: string (nullable = true)
 |-- Qualidade de vida: string (nullable = true)
 |-- Remuneração e benefícios: string (nullable = true)
 |-- reviews_count: string (null

employer_name,Cultura e valores,employer_website,Alta liderança,culture_count,employer_revenue,Oportunidades de carreira,Perspectiva positiva da empresa(%),Geral,salaries_count,match_percent,employer_founded,employer_headquarters,employer_industry,CNPJ,url,Recomendam para outras pessoas(%),Segmento,Qualidade de vida,Remuneração e benefícios,reviews_count,benefits_count,Nome,Diversidade e inclusão
Apex Group,2.9,null,2.9,281,null,3.2,53.0,3.2,592,70,null,null,null,655522,"https://www.glassdoor.com.br/Vis%C3%A3o-geral/Trabalhar-na-Apex-Group-EI_IE764729.13,23.htm",52.0,null,3.0,2.7,450,74,APE POUPEX,3.5
Banco Alfa,3.2,null,2.8,74,null,2.7,47.0,3.5,271,67,null,null,null,58160789,"https://www.glassdoor.com.br/Vis%C3%A3o-geral/Trabalhar-na-Banco-Alfa-EI_IE817306.13,23.htm",68.0,null,3.1,3.8,175,105,SAFRA,3.2
China Construction Bank,3.0,null,2.9,194,null,2.8,42.0,3.4,720,77,null,null,null,7450604,"https://www.glassdoor.com.br/Vis%C3%A3o-geral/Trabalhar-na-China-Construction-Bank-EI_IE12617.13,36.htm",54.0,null,3.4,3.5,871,240,CHINA CONSTRUCTION BANK (BRASIL) BANCO MULTIPLO S/,3.0
Mercedes-Benz Mobility,4.1,null,3.7,90,null,3.6,51.0,4.1,513,73,null,null,null,60814191,"https://www.glassdoor.com.br/Vis%C3%A3o-geral/Trabalhar-na-Mercedes-Benz-Mobility-EI_IE38563.13,35.htm",87.0,null,4.4,4.0,282,118,MERCEDES-BENZ,4.0
Sumitomo Mitsui Financial Group,3.2,null,3.1,246,null,3.1,53.0,3.4,1300,65,null,null,null,60518222,"https://www.glassdoor.com.br/Vis%C3%A3o-geral/Trabalhar-na-Sumitomo-Mitsui-Financial-Group-EI_IE256138.13,44.htm",56.0,null,3.4,3.3,903,290,BANCO SUMITOMO MITSUI BRASIL S.A.,3.3
BNP Paribas,3.7,null,3.4,4100,null,3.5,63.0,3.8,20000,100,null,null,null,null,"https://www.glassdoor.com.br/Vis%C3%A3o-geral/Trabalhar-na-BNP-Paribas-EI_IE10342.13,24.htm",77.0,S3,3.8,3.4,13000,3600,BNP PARIBAS,4.0
BTG Pactual,3.7,null,3.7,683,null,4.2,73.0,4.0,2800,100,null,null,null,null,"https://www.glassdoor.com.br/Vis%C3%A3o-geral/Trabalhar-na-BTG-Pactual-EI_IE411540.13,24.htm",78.0,S1,2.8,4.4,1600,635,BTG PACTUAL,3.5
Banco Alfa,3.2,null,2.8,74,null,2.7,47.0,3.5,271,100,null,null,null,null,"https://www.glassdoor.com.br/Vis%C3%A3o-geral/Trabalhar-na-Banco-Alfa-EI_IE817306.13,23.htm",68.0,S3,3.1,3.8,175,105,ALFA,3.2
Banco BMG,3.9,null,3.6,232,null,3.5,63.0,4.1,704,100,null,null,null,null,"https://www.glassdoor.com.br/Vis%C3%A3o-geral/Trabalhar-na-Banco-BMG-EI_IE2482923.13,22.htm",79.0,S3,3.7,4.2,445,277,BMG,4.1
Banco Bradesco,3.8,null,3.4,3300,null,3.8,66.0,4.2,19000,100,null,null,null,null,"https://www.glassdoor.com.br/Vis%C3%A3o-geral/Trabalhar-na-Banco-Bradesco-EI_IE10997.13,27.htm",81.0,S1,3.4,4.3,11000,3200,BRADESCO,3.9


In [0]:
try: 
    logger.info(f"Salvando {df_bronze_bancos}, {df_unificado}, {df_bronze_reclamacoes} na camada Bronze .JSON")
    df_bronze_bancos.write.format("json").mode("overwrite").save("/FileStore/glassdoor/bronze/bancos/")
    df_unificado.write.format("json").mode("overwrite").save("/FileStore/glassdoor/bronze/empregados/")
    df_bronze_reclamacoes.write.format("json").mode("overwrite").save("/FileStore/glassdoor/bronze/reclamacoes/")
    logger.info(f"Salvo com sucesso!")
except AnalysisException as e:
            logger.error(f"Erro ao salvar os arquivos JSON: {str(e)}")

In [0]:
display(df_bronze_bancos)
display(df_unificado)
display(df_bronze_reclamacoes)

Segmento,CNPJ,Nome
S1,0,BANCO DO BRASIL - PRUDENCIAL
S1,60746948,BRADESCO - PRUDENCIAL
S1,30306294,BTG PACTUAL - PRUDENCIAL
S1,360305,CAIXA ECONOMICA FEDERAL - PRUDENCIAL
S1,60872504,ITAU - PRUDENCIAL
S1,90400888,SANTANDER - PRUDENCIAL
S2,92702067,BANRISUL - PRUDENCIAL
S2,7237373,BANCO DO NORDESTE DO BRASIL S.A. - PRUDENCIAL
S2,33657248,BNDES - PRUDENCIAL
S2,33479023,CITIBANK - PRUDENCIAL


employer_name,Cultura e valores,employer_website,Alta liderança,culture_count,employer_revenue,Oportunidades de carreira,Perspectiva positiva da empresa(%),Geral,salaries_count,match_percent,employer_founded,employer_headquarters,employer_industry,CNPJ,url,Recomendam para outras pessoas(%),Segmento,Qualidade de vida,Remuneração e benefícios,reviews_count,benefits_count,Nome,Diversidade e inclusão
Apex Group,2.9,null,2.9,281,null,3.2,53.0,3.2,592,70,null,null,null,655522,"https://www.glassdoor.com.br/Vis%C3%A3o-geral/Trabalhar-na-Apex-Group-EI_IE764729.13,23.htm",52.0,null,3.0,2.7,450,74,APE POUPEX,3.5
Banco Alfa,3.2,null,2.8,74,null,2.7,47.0,3.5,271,67,null,null,null,58160789,"https://www.glassdoor.com.br/Vis%C3%A3o-geral/Trabalhar-na-Banco-Alfa-EI_IE817306.13,23.htm",68.0,null,3.1,3.8,175,105,SAFRA,3.2
China Construction Bank,3.0,null,2.9,194,null,2.8,42.0,3.4,720,77,null,null,null,7450604,"https://www.glassdoor.com.br/Vis%C3%A3o-geral/Trabalhar-na-China-Construction-Bank-EI_IE12617.13,36.htm",54.0,null,3.4,3.5,871,240,CHINA CONSTRUCTION BANK (BRASIL) BANCO MULTIPLO S/,3.0
Mercedes-Benz Mobility,4.1,null,3.7,90,null,3.6,51.0,4.1,513,73,null,null,null,60814191,"https://www.glassdoor.com.br/Vis%C3%A3o-geral/Trabalhar-na-Mercedes-Benz-Mobility-EI_IE38563.13,35.htm",87.0,null,4.4,4.0,282,118,MERCEDES-BENZ,4.0
Sumitomo Mitsui Financial Group,3.2,null,3.1,246,null,3.1,53.0,3.4,1300,65,null,null,null,60518222,"https://www.glassdoor.com.br/Vis%C3%A3o-geral/Trabalhar-na-Sumitomo-Mitsui-Financial-Group-EI_IE256138.13,44.htm",56.0,null,3.4,3.3,903,290,BANCO SUMITOMO MITSUI BRASIL S.A.,3.3
BNP Paribas,3.7,null,3.4,4100,null,3.5,63.0,3.8,20000,100,null,null,null,null,"https://www.glassdoor.com.br/Vis%C3%A3o-geral/Trabalhar-na-BNP-Paribas-EI_IE10342.13,24.htm",77.0,S3,3.8,3.4,13000,3600,BNP PARIBAS,4.0
BTG Pactual,3.7,null,3.7,683,null,4.2,73.0,4.0,2800,100,null,null,null,null,"https://www.glassdoor.com.br/Vis%C3%A3o-geral/Trabalhar-na-BTG-Pactual-EI_IE411540.13,24.htm",78.0,S1,2.8,4.4,1600,635,BTG PACTUAL,3.5
Banco Alfa,3.2,null,2.8,74,null,2.7,47.0,3.5,271,100,null,null,null,null,"https://www.glassdoor.com.br/Vis%C3%A3o-geral/Trabalhar-na-Banco-Alfa-EI_IE817306.13,23.htm",68.0,S3,3.1,3.8,175,105,ALFA,3.2
Banco BMG,3.9,null,3.6,232,null,3.5,63.0,4.1,704,100,null,null,null,null,"https://www.glassdoor.com.br/Vis%C3%A3o-geral/Trabalhar-na-Banco-BMG-EI_IE2482923.13,22.htm",79.0,S3,3.7,4.2,445,277,BMG,4.1
Banco Bradesco,3.8,null,3.4,3300,null,3.8,66.0,4.2,19000,100,null,null,null,null,"https://www.glassdoor.com.br/Vis%C3%A3o-geral/Trabalhar-na-Banco-Bradesco-EI_IE10997.13,27.htm",81.0,S1,3.4,4.3,11000,3200,BRADESCO,3.9


Ano,Trimestre,Categoria,Tipo,CNPJ IF,Instituição financeira,Índice,Quantidade de reclamações reguladas procedentes,Quantidade de reclamações reguladas - outras,Quantidade de reclamações não reguladas,Quantidade total de reclamações,Quantidade total de clientes – CCS e SCR,Quantidade de clientes – CCS,Quantidade de clientes – SCR,_c14
2022,3º,"Demais Bancos, Financeiras e Instituições de Pagamento",Conglomerado,,ABC-BRASIL (conglomerado),,4,7,3,14,50720,47485,6932,null
2022,3º,"Demais Bancos, Financeiras e Instituições de Pagamento",Conglomerado,,ABN AMRO (conglomerado),,0,0,1,1,0,0,,null
2022,3º,"Demais Bancos, Financeiras e Instituições de Pagamento",Banco/financeira,13140088,ACESSO SOLUÇÕES DE PAGAMENTO S.A.,"14,70",75,128,9,212,5098671,5098661,19396,null
2022,3º,"Demais Bancos, Financeiras e Instituições de Pagamento",Conglomerado,,AGIBANK (conglomerado),"96,00",182,262,95,539,1895683,1540134,1176067,null
2022,3º,"Demais Bancos, Financeiras e Instituições de Pagamento",Banco/financeira,36321990,"AGORACRED S/A SOCIEDADE DE CRÉDITO, FINANCIAMENTO E INVESTIMENTO",,4,4,2,10,534795,133,534662,null
2022,3º,"Demais Bancos, Financeiras e Instituições de Pagamento",Banco/financeira,27214112,"AL5 S.A. CRÉDITO, FINANCIAMENTO E INVESTIMENTO",,10,3,2,15,30959,19706,16841,null
2022,3º,"Demais Bancos, Financeiras e Instituições de Pagamento",Conglomerado,,ALFA (conglomerado),,23,48,9,80,436746,267423,170417,null
2022,3º,"Demais Bancos, Financeiras e Instituições de Pagamento",Banco/financeira,32778350,AME DIGITAL BRASIL INSTITUICAO DE PAGAMENTO LTDA,,6,31,90,127,0,0,0,null
2022,3º,"Demais Bancos, Financeiras e Instituições de Pagamento",Conglomerado,,ANDBANK (conglomerado),,0,2,2,4,4302,4302,29,null
2022,3º,"Demais Bancos, Financeiras e Instituições de Pagamento",Banco/financeira,19540550,ASAAS GESTÃO FINANCEIRA INSTITUIÇÃO DE PAGAMENTO S.A.,,8,42,5,55,404447,404447,,null
